#### 1. 라이브러리 설치 및 환경 설정
- 예시코드 : https://puleugo.tistory.com/10
- MediaPipe Studio : https://mediapipe-studio.webapps.google.com/home
- Python setup : https://developers.google.com/mediapipe/solutions/setup_python
- Python code : https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer/python
- Documention : https://developers.google.com/mediapipe
- land mark & Gesture classification bundle : https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer#gesture_classification_model_bundle

In [ ]:
!python.exe -m pip install --upgrade pip
!pip install opencv-python
!pip install mediapipe
!pip install --upgrade numpy

In [ ]:
# protobuf 설치
# upgrade
!pip install --upgrade protobuf

In [ ]:
# 설치 경로 확인 -> site-packages/google/protobuf/internal 
# 안에 있는 builder.py를 찾아서 바탕화면에 copy
!pip show protobuf

In [ ]:
# pip install protobuf==3.19.4 로 버전을 낮춘다
# builder.py를 다시 pip show protobuf로 경로를 찾아서
# site-packages/google/protobuf/internal에 넣어준다.
!pip install protobuf==3.19.4

#### 2. 카메라로부터 영상 가져오기

In [6]:
import cv2

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cv2.waitKey(33) < 0:
    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

#### 3. 손가락 랜드마크 감지
- 랜드마크 좌표 출력

In [9]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("카메라를 찾을 수 없습니다.")
      continue

    # 이미지 좌우로 뒤집기
    image = cv2.flip(image, 1)

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
        # 각 랜드마크의 좌표를 표시합니다.
        image_height, image_width, _ = image.shape
        for idx, landmark in enumerate(hand_landmarks.landmark):
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)
            cv2.putText(image, f'LM{idx} ({x}, {y})', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()


: 

#### 4. 제스처 감지
이 코드에서는 손의 검지만을 사용하여 커서 모드를 제어합니다. 검지의 두 번째 관절 (PIP)과 세 번째 관절 (DIP) 그리고 손목과의 상대적 위치를 비교하여 검지를 펼치고 있는지 여부를 확인합니다. 검지를 펼친 경우 "커서 모드입니다"가 화면 왼쪽 위에 표시됩니다. 검지를 접으면 커서 모드가 해제됩니다.

##### 커서 모드 진입

In [8]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
  cursor_mode = False  # 초기에는 커서 모드가 아님을 나타냅니다.

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("카메라를 찾을 수 없습니다.")
      continue

    # 이미지 좌우로 뒤집기
    image = cv2.flip(image, 1)

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
        # 검지와 중지 손가락 끝점 좌표를 가져옵니다.
        index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
        
        # 검지 손가락을 펼치고 나머지 손가락은 접었는지 확인합니다.
        if index_finger_tip.y < middle_finger_tip.y:
          cursor_mode = True  # 커서 모드로 전환합니다.
          cv2.putText(image, "cursor_mode", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        else:
          cursor_mode = False  # 커서 모드가 아님을 나타냅니다.

    cv2.imshow('MediaPipe Hands', image)
    # esc 키로 종료
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()


##### 커서모드 + 검지 트래킹 + 커서모드 종료 모션 추가

In [3]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
  cursor_mode = False  # 초기에는 커서 모드가 아님을 나타냅니다.

  cursor_position = None  # 커서 위치 초기화

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("카메라를 찾을 수 없습니다.")
      continue

    # 이미지 좌우로 뒤집기
    image = cv2.flip(image, 1)

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
        # 검지와 중지 손가락 끝점 좌표를 가져옵니다.
        index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
        
        # 검지 손가락을 펼치고 나머지 손가락은 접었는지 확인합니다.
        if index_finger_tip.y < middle_finger_tip.y:
          cursor_mode = True  # 커서 모드로 전환합니다.
          cursor_position = (int(index_finger_tip.x * image.shape[1]), int(index_finger_tip.y * image.shape[0]))
          cv2.putText(image, "Cursor Mode", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          cv2.circle(image, cursor_position, 10, (0, 0, 255), -1)  # 커서 위치에 원 그리기

          # 화살표 그리기
          cv2.arrowedLine(image, cursor_position, (cursor_position[0] + 50, cursor_position[1]), (255, 0, 0), 2)
          cv2.arrowedLine(image, cursor_position, (cursor_position[0], cursor_position[1] + 50), (0, 255, 0), 2)
          
        else:
          cursor_mode = False  # 커서 모드가 아님을 나타냅니다.
          cursor_position = None  # 커서 위치 초기화

    cv2.imshow('MediaPipe Hands', image)
    # esc 키로 종료
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()


##### 윈도우 커서 제어
OpenCV와 Mediapipe를 사용하여 손을 감지하고, 감지된 손의 위치를 기반으로 마우스 커서를 조작할 수 있습니다.

아래는 이를 구현하는 간단한 예제 코드입니다. 이 예제에서는 손의 중심을 사용하여 마우스 커서를 제어합니다

- pyautogui doc : https://wikidocs.net/85581

In [ ]:
!pip install pyautogui

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui

# Mediapipe 라이브러리를 사용하기 위해 모듈을 import 합니다.
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 웹캠에서 영상을 받아오기 위해 VideoCapture 객체를 생성합니다.
cap = cv2.VideoCapture(0)

# Hands 모델을 사용하여 손을 추적합니다.
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        # 웹캠으로부터 현재 프레임을 읽어옵니다.
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            break

        # 이미지를 좌우로 뒤집습니다.
        image = cv2.flip(image, 1)
        
        # Mediapipe 라이브러리에서 사용하는 RGB 형식으로 이미지를 변환합니다.
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Hands 모델을 사용하여 손을 감지합니다.
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # 손 중심의 x, y 좌표를 계산합니다.
                cx, cy = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * image.shape[1]), int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * image.shape[0])

                # 마우스 커서를 손 중심으로 이동시킵니다.
                pyautogui.moveTo(cx, cy)

                # 손을 화면에 그립니다.
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # 화면에 영상을 출력합니다.
        cv2.imshow('Hand Tracking', image)

        # 'esc' 키를 누르면 루프를 종료합니다.
        if cv2.waitKey(1) & 0xFF == 27:
            break

# 웹캠 리소스를 해제하고 모든 창을 닫습니다.
cap.release()
cv2.destroyAllWindows()

커서 모드

1. 커서의 이동은 검지를 폈을때만 가능하도록, 이때는 클릭되면 안됨
2. 검지와 중지를 펼쳐서 붙이면 클릭
3. 손가락을 모두 펼치면 더블클릭
4. 검지와 엄지를 붙이는 모양으로 화면 확대
5. 검지와 엄지를 펼치면 화면 축소
6. 주먹을 쥐면 커서모드를 종료시켜줘

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    cursor_mode = False
    clicking = False
    double_click = False
    zoom_in = False
    zoom_out = False

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            break

        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
                index_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]

                if all(finger.y > thumb_mcp.y for finger in [middle_finger_tip, ring_finger_tip, pinky_tip]):
                    cursor_mode = True
                    cursor_position = (int(index_finger_tip.x * image.shape[1]), int(index_finger_tip.y * image.shape[0]))
                    pyautogui.moveTo(cursor_position[0], cursor_position[1], duration=0.1)
                else:
                    cursor_mode = False

                if all(finger.x < index_finger_tip.x for finger in [middle_finger_tip, index_finger_tip]):
                    clicking = True
                else:
                    clicking = False

                if all(finger.x < thumb_mcp.x for finger in [index_finger_tip, middle_finger_tip, thumb_tip]):
                    double_click = True
                else:
                    double_click = False

                if np.linalg.norm(np.array([index_finger_tip.x, index_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) < 0.03:
                    zoom_in = True
                else:
                    zoom_in = False

                if np.linalg.norm(np.array([index_finger_tip.x, index_finger_tip.y]) - np.array([thumb_tip.x, thumb_tip.y])) > 0.1:
                    zoom_out = True
                else:
                    zoom_out = False

                if all(finger.x > index_finger_tip.x for finger in [middle_finger_tip, index_finger_tip]):
                    cursor_mode = False
                    clicking = False
                    double_click = False
                    zoom_in = False
                    zoom_out = False

                if cursor_mode:
                    cv2.circle(image, cursor_position, 10, (0, 0, 255), -1)
                    cv2.putText(image, "Cursor Mode", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    if cursor_position is not None:
                        arrow_end_x = int(cursor_position[0] + 50)
                        arrow_end_y = int(cursor_position[1] + 50)
                        cv2.arrowedLine(image, cursor_position, (arrow_end_x, cursor_position[1]), (255, 0, 0), 2)
                        cv2.arrowedLine(image, cursor_position, (cursor_position[0], arrow_end_y), (0, 255, 0), 2)

                if clicking:
                    pyautogui.click()

                if double_click:
                    pyautogui.doubleClick()

                if zoom_in:
                    pyautogui.scroll(1)

                if zoom_out:
                    pyautogui.scroll(-1)

                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
